# Create a Database

We will be working with Super Heroes to illustrate working with DBs to match the same example as SQLModel's [official docs](https://sqlmodel.tiangolo.com/).

The table we want to create will end up looking like this:

| id | name        | secret_name   | age |
|----|-------------|---------------|-----|
| 1  | Deadpond    | Dive Wilson   | null|
| 2  | Spider-Boy  | Pedro Parqueador | null|
| 3  | Rusty-Man   | Tommy Sharp   | 48  |

## Using raw SQL

Obviously, we _can_ use SQL directly, but this would only work in a SQL Client or as a single string, which won't give you any linting, type checking, auto-complete, etc:

In [ ]:
CREATE TABLE "hero" (
  "id"  INTEGER,
  "name"  TEXT NOT NULL,
  "secret_name" TEXT NOT NULL,
  "age" INTEGER,
  PRIMARY KEY("id")
);


## Use SQLModel

Instead, we can use an Object-Relational Mapping (ORM) which is a fancy way of saying, "Write SQL in a programming language", like Python!

> 💡 SQLModel is an ORM library that uses SQLAlchemy and Pydantic to work with databases

We will:

1. Define a table with SQLModel
2. Create a SQLite database and table with SQLModel

In [1]:
from typing import Optional

from sqlmodel import Field, SQLModel, create_engine


# 1. Define the SQLModel class, with table=True which means this model corresponds to a database table
class Hero(SQLModel, table=True):
    # 2. Define attributes corresponding to columns of the table
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    # 3. Optional, in this case, means that age can be an Integer or None (or NULL in SQL)
    age: Optional[int] = None

# 4. Define the database URL, in this case, a SQLite database
sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

# 5. Create the database engine, which is the object that handles the connection to the database
engine = create_engine(sqlite_url, echo=True) # echo=True prints things to the console so you can see what's happening under the hood

# 6. Create the database and table
SQLModel.metadata.create_all(engine)

2023-12-13 00:00:51,374 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 00:00:51,375 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2023-12-13 00:00:51,376 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-13 00:00:51,376 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2023-12-13 00:00:51,377 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-13 00:00:51,377 INFO sqlalchemy.engine.Engine 
CREATE TABLE hero (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	secret_name VARCHAR NOT NULL, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2023-12-13 00:00:51,378 INFO sqlalchemy.engine.Engine [no key 0.00025s] ()
2023-12-13 00:00:51,379 INFO sqlalchemy.engine.Engine COMMIT


## Download DB Browser for SQLite (free)

We can use DB Browser which is a free DB Client for working with SQLite databases.

It's completely free and can be installed from their [Official Downloads](https://sqlitebrowser.org/dl/) page.

1. Download DB Browser
2. Open DB Browser and click `Open Database`
3. Find the `database.db` file that was created from the cell above
4. Now you see the database and `hero` table!
5. Click the `Execute SQL` tab and run any SQL against it

```sql
select * from hero
```

> ❌ Nothing is returned... what gives??? We haven't created any heroes yet! You'll see that in the next notebook

## Refactor Data Creation

We'll restructure the code a bit to make it easier to **reuse**, **share**, and **test** by moving some things into functions. You could even put this whole thing into a python script to execute as needed!

In [ ]:
from typing import Optional

from sqlmodel import Field, SQLModel, create_engine


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None


sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)

# Changes here ⬇️

# 1. Create a function that creates the database and tables that can be reused
def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

# 2. This __main__ block will run when you execute the file as a Python script
if __name__ == "__main__":
    create_db_and_tables()